## Pre-requirement

Open an IB account. [IB website](https://www.interactivebrokers.ca/en/home.php)

## Step 1 - Install IB TWS (Trader Workstation)
- Download and insall [TWS software](https://www.interactivebrokers.ca/en/index.php?f=45276#tws-software)  

IB provides APIs for both TWS and IB GateWay. TWS is recommanded because you can have GUI to verify the placed orders. IB GateWay has not GUI. So you can only use it by API. IB Gateway is suitable when the robot is much stable.

## Step 2 - Install TWS API
- Download [TWS API package](https://www.interactivebrokers.ca/en/index.php?f=45481)  
- Unzip package, and cd IBJts/source/pythonclient  
- Install local package: python -m setup.py install 
- Check if ibapi package is installed: python -c 'import ibapi'


## Step 3 - Configure TWS
- Open TWS, login to <font color='red'>**paper account**</font>.
- File->Global Configuration->API->Settings
![](tws-config.png)

## Step 4 - Test TWS API

Please run each cell one by one

In [48]:
import time
import datetime
import threading

import ibapi
import ibapi.wrapper as wrapper
import ibapi.client as client
from ibapi.wrapper import EWrapper
from ibapi.client import EClient
from ibapi.contract import Contract
from ibapi.order import Order
from ibapi.ticktype import TickType, TickTypeEnum

from ibapi import comm  # read message returned from API
from ibapi.account_summary_tags import AccountSummaryTags
from ibapi.utils import iswrapper

In [49]:
def createContract(
    symbol,
    sec_type='STK',
    exch='SMART',
    prim_exch='ISLAND',
    currency='USD'
):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = sec_type
    contract.exchange = exch
    contract.primaryExch = prim_exch
    contract.currency = currency
    return contract

def createOrder(action, quantity, limit_price=None):

    def LimitOrder(action, quantity, limit_price):
        # ! [limitorder]
        order = Order()
        order.action = action
        order.orderType = "LMT"
        order.totalQuantity = quantity
        order.lmtPrice = limit_price
        # ! [limitorder]
        return order

    def MarketOrder(action, quantity):
        # ! [market]
        order = Order()
        order.action = action
        order.orderType = "MKT"
        order.totalQuantity = quantity
        # ! [market]
        return order

    if limit_price is None:
        return MarketOrder(action, quantity)
    else:
        return LimitOrder(action, quantity, limit_price)


In [50]:
class IBClient(EClient):  # For sending msg
    def __init__(self, wrapper):
        EClient.__init__(self, wrapper)

class IBWrapper(EWrapper):  # For receiving msg
    def __init__(self):
        EWrapper.__init__(self)

class IBClientApp(IBWrapper, IBClient):
    def __init__(self, IP, port, clientId):
        IBWrapper.__init__(self)
        IBClient.__init__(self, wrapper=self)
        self.IP = IP
        self.port = port
        self.clientId = clientId
        self.info = {'positions': []}
        self.history_data = {}
        self.order_record = {}
        self.permId2ord = {}

    def getConnection(self):
        self.connect(self.IP, self.port, self.clientId)
        self.reqIds(-1)
        self.getMessage(1)
    
    def getNextValidId(self):
        self.valid_id += 1
        return self.valid_id
    
    @iswrapper
    def nextValidId(self, orderId):
        super().nextValidId(orderId)
        print(f'First Valid orderId is: {orderId}')
        self.valid_id = orderId

    @iswrapper
    def error(self, reqId, errorCode, errorString):
        super().error(reqId, errorCode, errorString)
        if int(errorCode) >= 2000:
            return
        print('| Server return an error! reqId: %s, errorCode:%s, msg:%s' % (
            reqId, errorCode, errorString))

    def getAccInfo(self):
        self.info = {'positions': []}
        self.reqAccountSummary(102, "All", AccountSummaryTags.AllTags)
        self.reqPositions()
        time.sleep(2)
        self.cancelAccountSummary(102)
        self.getMessage(1)
        return self.info

    @iswrapper
    def accountSummary(self, reqId: int, account: str, tag: str, value: str,
                       currency: str):
        super().accountSummary(reqId, account, tag, value, currency)
        if (tag == 'TotalCashValue'):
            self.info['cash'] = value
        if (tag == 'NetLiquidation'):
            self.info['total'] = value

    @iswrapper
    def accountSummaryEnd(self, reqId: int):
        super().accountSummaryEnd(reqId)
        return

    @iswrapper
    def position(self, account, contract, position, avgCost):
        super().position(account, contract, position, avgCost)
        tmp = [contract.symbol, contract.secType, contract.currency, position, avgCost]
        self.info['positions'].append(tmp)

    @iswrapper
    def positionEnd(self):
        super().positionEnd()
        return

    def getHistoryData(self, reqId, symbol, queryTime, lastFor='10000 S', timeGap='5 secs'):
        contract = createContract(symbol, 'STK', 'SMART', 'SMART', 'USD')
        self.reqHistoricalData(
            reqId, contract, queryTime,
            lastFor, timeGap, 'TRADES', 1, 1, False, []
        )

    def getWatchListData(self, reqId, symbol):
        contract = createContract(symbol, 'STK', 'SMART', 'SMART', 'USD')
        self.reqMktData(
            reqId, contract,
            "", False, False, []
        )

    @iswrapper
    def historicalData(self, reqId, date, open, high,
                       low, close, volume, barCount,
                       WAP, hasGaps):
        super().historicalData(reqId, date, open, high, low, close, volume,
                               barCount, WAP, hasGaps)

        if reqId not in self.history_data.keys():
            self.history_data[reqId] = []

        single_row = '%s,%s,%s,%s,%s,%s\n' % (
            date, open, high, low, close, volume
        )
        self.history_data[reqId].append(single_row)

    @iswrapper
    def historicalDataEnd(self, reqId: int, start: str, end: str):
        super().historicalDataEnd(reqId, start, end)

    def sendOrderToServer(
        self,
        symbol,
        quantity,
        sec_type='STK',
        primary_exch='SMART',
        price=None
    ):

        contract = createContract(
            symbol,
            sec_type,
            'SMART',
            primary_exch,
            'USD'
        )
        action = "BUY" if quantity > 0 else "SELL"
        order = createOrder(action, abs(quantity), price)
        orderId = self.getNextValidId()

        print('|- Place order. ID is %d' % orderId)
        self.placeOrder(orderId, contract, order)
        print(self.order_record)
        self.order_record[orderId] = [symbol, action, False]

    @iswrapper
    def orderStatus(self, orderId, status, filled,
                    remaining, avgFillPrice, permId,
                    parentId, lastFillPrice, clientId,
                    whyHeld, mktCapPrice):
        super().orderStatus(orderId, status, filled, remaining,
                            avgFillPrice, permId, parentId,
                            lastFillPrice, clientId, whyHeld, mktCapPrice)

        if status != 'Filled' or self.order_record[orderId][2]:
            return
        symbol = self.order_record[orderId][0]
        action = self.order_record[orderId][1]
        self.order_record[orderId][2] = True

        try:
            msg = '| %s Filled! %s quantity:%d avgPrice:%.2f Total:%.2f\n' % (
                    time.strftime('%Y%m%d %H:%M:%S'),
                    action, filled, avgFillPrice,
                    filled * avgFillPrice
                )
            print(msg)
            self.logger.log(symbol, msg)
        except Exception:
            print('| Error in logger!')

    @iswrapper
    def openOrder(self, orderId, contract, order, orderState):
        super().openOrder(orderId, contract, order, orderState)
        # OpenOrder. ID: 2 UVXY STK @ SMART : BUY MKT 10.0 PreSubmitted
        print("OpenOrder. ID:", orderId, contract.symbol, contract.secType,
              "@", contract.exchange, ":", order.action, order.orderType,
              order.totalQuantity, orderState.status)
        order.contract = contract
        self.permId2ord[order.permId] = order

    @iswrapper
    def openOrderEnd(self):
        # ! [openorderend]
        super().openOrderEnd()
        print("OpenOrderEnd")
        # ! [openorderend]
        print("Received %d openOrders" % len(self.permId2ord))

    def getMessage(self, wait=3):
        time.sleep(wait)
        while not self.msg_queue.empty():
            text = self.msg_queue.get(block=True, timeout=0.2)
            fields = comm.read_fields(text)
            self.decoder.interpret(fields)
    
    def tickPrice(self, reqId, tickType, price: float, attrib):
        super().tickPrice(reqId, tickType, price, attrib)
        print("TickPrice. TickerId:", reqId, "tickType:", tickType, "Price:", price, "CanAutoExecute:", attrib.canAutoExecute,"PastLimit:", attrib.pastLimit, end=' ')
        if tickType == TickTypeEnum.BID or tickType == TickTypeEnum.ASK:
            print("PreOpen:", attrib.preOpen)
        else:
            print()

### Connect and run app

In [51]:
app = IBClientApp('127.0.0.1', 7497, clientId=0)
app.getConnection()
app.reqMarketDataType(3)  # set to delayed stream data
ib_thread = threading.Thread(name='ib_thread', target=app.run)
ib_thread.start()

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


OpenOrder. ID: 5 BA STK @ SMART : BUY MKT 1.0 PreSubmitted
OpenOrder. ID: 9 BA STK @ SMART : BUY MKT 1.0 PreSubmitted
OpenOrderEnd
Received 2 openOrders
First Valid orderId is: 10
First Valid orderId is: 10


In [52]:
app.getAccInfo()

{'positions': [['IBKR', 'STK', 'USD', -1.0, 53.757],
  ['BA', 'STK', 'USD', 316.0, 212.52432815],
  ['AAPL', 'STK', 'USD', 800.0, 136.1100776]],
 'total': '1009942.92',
 'cash': '824204.23'}

In [53]:
app.getWatchListData(app.getNextValidId(), "GOEV")

ERROR 11 10167 Requested market data is not subscribed. Displaying delayed market data.


TickPrice. TickerId: 11 tickType: 68 Price: 9.74 CanAutoExecute: False PastLimit: False 
TickPrice. TickerId: 11 tickType: 72 Price: 10.18 CanAutoExecute: False PastLimit: False 
TickPrice. TickerId: 11 tickType: 73 Price: 9.54 CanAutoExecute: False PastLimit: False 
TickPrice. TickerId: 11 tickType: 75 Price: 10.33 CanAutoExecute: False PastLimit: False 
TickPrice. TickerId: 11 tickType: 66 Price: -1.0 CanAutoExecute: False PastLimit: False 
TickPrice. TickerId: 11 tickType: 67 Price: -1.0 CanAutoExecute: False PastLimit: False 
TickPrice. TickerId: 11 tickType: 76 Price: 0.0 CanAutoExecute: False PastLimit: False 


In [54]:
queryTime = (datetime.datetime.today() - datetime.timedelta(days=1)).strftime("%Y%m%d %H:%M:%S")
app.getHistoryData(app.getNextValidId(), 'AAPL', queryTime, '100 s', '1 secs')

ERROR 12 162 Historical Market Data Service error message:No market data permissions for ISLAND STK


| Server return an error! reqId: 12, errorCode:162, msg:Historical Market Data Service error message:No market data permissions for ISLAND STK


In [55]:
app.sendOrderToServer('BA', 1)

|- Place order. ID is 13
{}


ERROR 13 399 Order Message:
BUY 1 BA NYSE


| Server return an error! reqId: 13, errorCode:399, msg:Order Message:
BUY 1 BA NYSE
OpenOrder. ID: 13 BA STK @ SMART : BUY MKT 1.0 PreSubmitted


### <font color='red'>**Must** disconnect</font> before you leave, otherwise next time connection may fail.

In [56]:
app.disconnect()